# <font color= 'blue'> Poisson Image Editing</font>

## Seamless Image Cloning using openCV

#### A notebook by *Dhruv Ilesh Shah*

***Packages Used: *** Python 2.7, openCV 3.1.0, `gimp 2.8` (recommended)

This notebook demonstrates how the Poisson Solver can be used to seamlessly clone an image(or a portion) onto another.
We begin by importing required packages.

In [1]:
import cv2
import numpy as np

For applying the algorithm, you will need the cloned image and a mask that identifies the cloned portion of the image. The purpose of the mask will clear in the subsequent steps.  
***How to obtain the mask?*** *This method needs improvement - suggestions welcome!*
* Given that you clone the image on your system, I recommend you use `gimp` for the purpose. Open `GIMP Image Editor`.
* Open the image on which cloning must occur, the destination image.
* In another window, open the source, whose portion is to be cloned. Using the select tool, select and copy the portion.
* Paste the portion on the destination image. You will see that a new layer with the pasted portion will be created. Using the `Transform` tool, move the portion to desired location and scale accordingly, using the `Scale` tool.
* Export this image and save your cloned file as a `jpeg` or `png` file. Now we must make the mask.
* Create a new `background layer` and place it below the pasted layer. Also, hide the destination image layer. Select the bg-layer and color it black using the Color Tool.
* You will now have the cloned portion on a black background. Using the Thresholding tool (or on `cv2`), threshold all pixels above 1 as 255.(*This should work unless your image has a pixel value 0, in which case you may have to use floodFill or explicitly modify the mask file*).
* Export this file as the mask image in the `jpeg` or `png` format.

Let's now begin with the algorithm implementation.

In [2]:
im_cloned = cv2.imread("stinkbug_ubuntu_cloned.png",  cv2.IMREAD_GRAYSCALE)
im_mask = cv2.imread("stinkbug_ubuntu_mask.png", cv2.IMREAD_GRAYSCALE)

it = 100; # Set number of iterations

After importing the image and mask, we create a temporary image to store the current value of pixels as the new ones are being calculated. This is because we want to simultaneously update values. * The iterative method used is called the ** Jacobi's Method. ** *

In [4]:
im_temp = im_cloned.copy()
im_seamless = im_temp.copy()
sigma = []
for i in range(im_cloned.shape[0]):
        for j in range(im_cloned.shape[1]):
            if (im_mask[i,j]==255):
                sigma.append([i,j])

for a in range(it):
    for [i,j] in sigma:
        term = 10000;
        term = term + im_seamless[i+1, j]+im_seamless[i-1, j]+im_seamless[i, j-1]+im_seamless[i, j+1]
        if(im_mask[i-1, j]==255):
            term = term + im_cloned[i,j]-im_cloned[i-1,j]
        if(im_mask[i+1, j]==255):
            term = term + im_cloned[i,j]-im_cloned[i+1,j]
        if(im_mask[i, j+1]==255):
            term = term + im_cloned[i,j]-im_cloned[i,j+1]
        if(im_mask[i, j-1]==255):
            term = term + im_cloned[i,j]-im_cloned[i,j-1]
        im_temp[i,j] = (term-10000)/4
    im_seamless = im_temp.copy()
    print a

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46


KeyboardInterrupt: 

As is visible from the code, we scan the whole cloned image for points where the corresponding value of the mask is 255 (*white*), and update `im_temp` as suggested in the algorithm. The gradient for the source image is defined as zero at the boundary points.
*Note that the pixel values in cv2 are of `ubyte_scalar` type and hence don't support addition due to overflow. `10000` is a random large number that helps maintain the variable `term` as a normal integer and hence allow addition without overflow*

In [ ]:
cv2.imshow("Cloning", im_cloned)
cv2.imshow("Seamless Cloning", im_seamless); cv2.waitKey(0); cv2.destroyAllWindows()

#### Thank You!